In [1]:
import pandas as pd
import numpy as np

from feature_extraction import person_finder, org_finder, state_agg, imputed_600, method_flagger

In [2]:
polls = pd.read_csv('../data/raw-polls.csv')
pollster_ratings = pd.read_csv('../data/pollster-ratings.csv')

## Data Cleaning

Before merging our data let's take a look at if we have any missing values.

In [3]:
polls.loc[:, polls.isna().sum() > 0].columns 

Index(['partisan', 'cand3_pct', 'bias', 'comment'], dtype='object')

Partisan, cand3_pct, and comment having NAs is reasonable but bias should not have any. It is possible that 538 has a reason for excluding these bias calculations but there is none listed and the data dictionary notes the exact calculation for bias so we will fix this now to make our plots are more accurate.

In [4]:
polls['calc_bias'] = polls['margin_poll'] - polls['margin_actual'] # same formula as data dictionary
np.mean(polls['error'] - np.abs(polls['calc_bias'])) # about as accurate as we can get

-9.27550055675687e-07

In [5]:
pollster_ratings.loc[:, pollster_ratings.isna().sum() > 0].columns

Index(['Mean-Reverted Bias', 'Bias', 'House Effect',
       'Average Distance from Polling Average (ADPA)'],
      dtype='object')

In [6]:
pollster_ratings[pollster_ratings.loc[:, pollster_ratings.isna().sum() > 0].columns].isna().sum()

Mean-Reverted Bias                               44
Bias                                             44
House Effect                                     62
Average Distance from Polling Average (ADPA)    108
dtype: int64

Our data here is incomplete, for the bias variables those are due to not enough polls to analyze bias on 538's part. House Effect requires a comparison to other partisan polls in the same race so these are likely NA's for a valid reason. ADPA however is a bit troubling so let's analyze that more closely.

In [7]:
pollster_ratings[pollster_ratings['Average Distance from Polling Average (ADPA)'].isna()]['Polls Analyzed'].describe()

count    108.000000
mean       2.277778
std        2.629215
min        1.000000
25%        1.000000
50%        1.500000
75%        2.000000
max       18.000000
Name: Polls Analyzed, dtype: float64

538 states their process for calculating ADPA is: we start by calculating how much the pollster’s average poll differs from the average of previous polls of that race — specifically, polls whose median field date was at least three days earlier and it is weighted based on the square root of the number of other polls in the field for each race.([source](https://fivethirtyeight.com/methodology/how-our-pollster-ratings-work/)) These NA values should be caused by races where there weren't enough other polls in the field, likely 0 previous polls. We will proceed as is because we do not intend to use any of these variables.

In [8]:
# Merging our dataframes for ease of use
df = pd.merge(left = polls, right = pollster_ratings, 
              how = 'left', 
              left_on = 'pollster_rating_id', right_on = 'Pollster Rating ID'
              )

In [9]:
df.loc[:, df.isna().sum() > 0].columns # Our merge caused new missing values so let's ensure there's no issue

Index(['partisan', 'cand3_pct', 'bias', 'comment', 'Rank', 'Pollster',
       'Pollster Rating ID', 'Polls Analyzed', 'AAPOR/Roper', 'Banned by 538',
       'Predictive Plus-Minus', '538 Grade', 'Mean-Reverted Bias',
       'Races Called Correctly', 'Misses Outside MOE', 'Simple Average Error',
       'Simple Expected Error', 'Simple Plus-Minus', 'Advanced Plus-Minus',
       'Mean-Reverted Advanced Plus-Minus', '# of Polls for Bias Analysis',
       'Bias', 'House Effect', 'Average Distance from Polling Average (ADPA)',
       'Herding Penalty'],
      dtype='object')

In [10]:
# First we gather the missing IDs
missing = [] 
for id in polls['pollster_rating_id'].unique():
    if int(id) not in list(df['Pollster Rating ID']):
        missing.append(id)

# Then we gather the names of the pollsters
missing_grades = []
for id in missing:
    for row in polls[polls['pollster_rating_id'] == id].values:
        if row[8] not in missing_grades:
            missing_grades.append(row[8])

# Then we count how many polls total are effected
for pollster in missing_grades:
    print(pollster, polls[polls['pollster'] == pollster].shape[0])

University of Maryland 1
Brigham Young University 4
Research America Inc. 3
Insights West 9
Sacred Heart University 2
Hofstra University 2


So, a number of our polls were done by pollsters that are not in our pollster ratings. This is unexpected and unfortunate but we don't have any way to work around this issue. Once we reach the modelling stage we will be removing these polls because we don't have information on them and we have no reasonable way to impute their values. It is somewhat troubling that 538 doesn't have these rated but we don't have any means to fix that.

## Feature Extraction

First, let's extract some features from the comment and methodology columns. Additionally, we will be cleaning the state column slightly to account for Maine and Nevada separate designations due to their unique presidential voting situation ([source](https://www.smithsonianmag.com/smart-news/why-do-maine-and-nebraska-split-their-electoral-votes-180976219/)). 

In [11]:
# These functions are defined in feature_extraction.py but basic descriptions are provided below

# Marks whether an organization is mentioned in the comment
# ie. 'for New York Daily News'
df['org'] = df['comment'].apply(
    lambda x: org_finder(x) if x is not np.nan else 0
    )

# Marks whether a person was mentioned in the comment
# ie. 'for Charles E. Schumer'
df['person'] = df['comment'].apply(
    lambda x: person_finder(x) if x is not np.nan else 0
    )

# Marks whether unspecified is in the comment
# ie. for unspecified Democratic sponsor
df['anon'] = df['comment'].str.contains('unspecified').apply(
    lambda x: 1 if x == True else 0
    )

# Marks whether poll is of 'voters' or 'registered voters'
# ie. among 'voters' || among registered voters
df['registered_voters'] = df['comment'].str.contains(
    r"registered voters|'voters'"
    ).apply(
        lambda x: 1 if x == True else 0
        ) # we are counting 'voters' as registered voters and not likely voters

# Marks whether the comment notes the poll is an average of multiple polls/models
# ie. average of multiple versions or turnout models listed in poll
df['averaged'] = df['comment'].str.contains('average').apply(
    lambda x: 1 if x == True else 0
    )

# Maps polls without partisan designations as No Party Listed (NPL)
# We also have a single 'IND' poll which is short for independent.
# This could refer to either an independent candidate or the Independent
# Party (a fringe far-right party). See below for references on this race
# but it seems like IND refers to an independent candidate in this case.
df['partisan'] = df['partisan'].map(
    {
        'D' : 'D',
        'R' : 'R',
        np.nan : 'NPL',
        'IND' : 'NPL' 
    }
)

# Gathers state designation from the location column which normally includes 
# district where applicable. The denotation 'agg' refers to us aggregating the
# Maine and Nevada districts that vote separately into the state as a whole for
# ease of analysis. 
# (https://www.smithsonianmag.com/smart-news/why-do-maine-and-nebraska-split-their-electoral-votes-180976219/)
df['state_agg'] = df['location'].apply(state_agg)

# Denotes polls with unspecified sample sizes that were imputed to be 600
# per 538's standard practice. A number of these polls had sample sizes
# that were not 600 and it is unclear why that is, this column only flags
# polls where the comment says the sample size was imputed and the listed 
# sample size is 600. Our suspicion is that polls with this comment and 
# non-600 sample sizes were polls that were originally imputed and later
# the correct value was added in but the comment was never changed.
df['imputed_600'] = imputed_600(df)

# Calculates the difference between median field date of the poll and 
# the election date. The values range from 1 day to 21 days as 538 only
# includes the 3 weeks before the election in this dataset.
df['days_bt_polldate_election'] = (pd.to_datetime(df['electiondate']) - pd.to_datetime(df['polldate'])).dt.days

# Creates flag variables from the methodology column.
# A poll can have multiple methodologies.
# ie. IVR/Online --> IVR:1, Online:1, Live Phone:0 (not exhaustive just demonstrative of the concept)
df = pd.merge(df, 
              pd.DataFrame(method_flagger(df['methodology'])), 
              left_index=True, 
              right_index=True)

In [12]:
polls[polls['partisan'] == 'IND'] # https://en.wikipedia.org/wiki/2010_Maine_gubernatorial_election
                                  # https://en.wikipedia.org/wiki/Eliot_Cutler

,poll_id,question_id,race_id,year,race,location,type_simple,type_detail,pollster,pollster_rating_id,...,electiondate,cand1_actual,cand2_actual,margin_actual,error,bias,rightcall,advancedplusminus,comment,calc_bias
5093,53001,83867,1292,2010,2010_Gov-G_ME,ME,Gov-G,Gov-G,Frederick Polls,108,...,11/2/2010,43.0,31.8,11.2,5.2,NaN,1.0,1.103192,for Eliot R. Cutler,-5.2


### Outputting Cleaned Dataframe with extracted Features

In [13]:
df.to_csv('../data/raw-polls-updated.csv', index = False)